# Preliminaries 

In [1]:
!rm -rf ./*
!git clone https://github.com/cosmolejo/MLDL23-FL-project
!mv MLDL23-FL-project/ MLDL23_FL_project/

Cloning into 'MLDL23-FL-project'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 67 (delta 5), reused 4 (delta 4), pack-reused 50
Unpacking objects: 100% (67/67), 98.32 KiB | 2.14 MiB/s, done.


In [ ]:

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for i,filename in enumerate(filenames):
#         if i%10==1:
#             print(os.path.join(dirname, filename))


# Libraries

In [5]:
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from sklearn.model_selection import train_test_split
import json
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
def data_parser(df):
    """
    takes a dataframe sorted by writers and unpacks the data
    return: dataframe with two columns [img,class]
    """
    transpose = df.T
    out_dict = dict()
    idx = 0
    for _, row in transpose.iterrows():
        row_x = row.x
        for i,val in enumerate(row.y):
            out_dict[idx] =[  np.array(row_x[i],dtype=np.float16),val]
            idx+=1
    out_df = pd.DataFrame(out_dict).T
    out_df = out_df.rename(columns={0:'img',
                          1:'class'})
    return out_df

In [7]:
df = pd.DataFrame()

for dirname, _, filenames in os.walk('/kaggle/input/femnist/all_data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        data = json.load(open(os.path.join(dirname, filename)))
        
        temp_df = pd.DataFrame(data['user_data'])
        temp_df = temp_df.reset_index(drop=True)
        df = pd.concat([df, temp_df], axis=1) # ignore_index=True
df = df.rename(index={0: "x", 1: "y"})
df

/kaggle/input/femnist/all_data/all_data_17.json
/kaggle/input/femnist/all_data/all_data_23.json
/kaggle/input/femnist/all_data/all_data_20.json
/kaggle/input/femnist/all_data/all_data_27.json
/kaggle/input/femnist/all_data/all_data_22.json
/kaggle/input/femnist/all_data/all_data_28.json
/kaggle/input/femnist/all_data/all_data_2.json
/kaggle/input/femnist/all_data/all_data_12.json
/kaggle/input/femnist/all_data/all_data_16.json
/kaggle/input/femnist/all_data/all_data_25.json
/kaggle/input/femnist/all_data/all_data_7.json
/kaggle/input/femnist/all_data/all_data_24.json
/kaggle/input/femnist/all_data/all_data_14.json
/kaggle/input/femnist/all_data/all_data_10.json
/kaggle/input/femnist/all_data/all_data_13.json
/kaggle/input/femnist/all_data/all_data_11.json
/kaggle/input/femnist/all_data/all_data_5.json
/kaggle/input/femnist/all_data/all_data_9.json
/kaggle/input/femnist/all_data/all_data_32.json
/kaggle/input/femnist/all_data/all_data_1.json
/kaggle/input/femnist/all_data/all_data_6.jso

,f2507_78,f2149_81,f2105_51,f2308_56,f2111_87,f2235_81,f2216_66,f2211_73,f2184_98,f2121_67,...,f1711_35,f1963_27,f1997_00,f1922_38,f1932_09,f1595_34,f1981_44,f1609_27,f1672_35,f1553_24
0,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",...,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
1,"[59, 59, 42, 53, 51, 56, 37, 49, 46, 47, 44, 6...","[37, 37, 45, 51, 59, 56, 40, 53, 46, 60, 52, 5...","[7, 7, 1, 3, 1, 9, 8, 7, 2, 1, 6, 3, 2, 6, 4, ...","[1, 1, 7, 7, 4, 9, 8, 5, 9, 9, 8, 7, 4, 1, 0, ...","[9, 9, 7, 8, 6, 2, 4, 8, 4, 0, 3, 9, 4, 6, 2, ...","[31, 31, 18, 15, 18, 12, 13, 25, 32, 23, 10, 1...","[24, 24, 22, 35, 32, 26, 28, 21, 12, 25, 10, 3...","[19, 19, 33, 31, 24, 16, 11, 29, 30, 28, 21, 1...","[42, 42, 61, 48, 46, 56, 41, 41, 54, 43, 37, 3...","[6, 6, 1, 3, 7, 4, 2, 9, 1, 2, 5, 3, 8, 6, 7, ...",...,"[2, 2, 7, 6, 2, 7, 4, 5, 2, 4, 3, 6, 3, 2, 0, ...","[3, 3, 7, 8, 8, 6, 5, 6, 3, 4, 6, 4, 8, 0, 0, ...","[6, 6, 9, 7, 2, 0, 4, 5, 2, 7, 7, 9, 5, 6, 4, ...","[44, 44, 58, 61, 54, 38, 53, 48, 39, 42, 56, 4...","[31, 31, 22, 12, 34, 13, 15, 30, 10, 18, 11, 2...","[9, 9, 8, 1, 2, 8, 2, 4, 3, 1, 4, 9, 8, 5, 2, ...","[23, 23, 19, 17, 13, 32, 25, 26, 21, 27, 35, 2...","[50, 50, 58, 45, 57, 49, 61, 36, 44, 53, 39, 4...","[17, 17, 26, 22, 14, 18, 18, 25, 20, 11, 31, 2...","[22, 22, 17, 28, 21, 33, 32, 14, 29, 11, 19, 2..."


In [9]:
batch = data_parser(df)
batch.head()

,img,class
0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",59
1,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",59
2,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",42
3,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",53
4,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",51


In [10]:
#release memory
del df

In [12]:
from torch.utils.data import Dataset
from torchvision import transforms
import MLDL23_FL_project.datasets.ss_transforms as tr
from torch import from_numpy
from matplotlib import cm
from PIL import Image
IMAGE_SIZE = 28
import os
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
# ])
convert_tensor = transforms.ToTensor()

class Femnist(Dataset):

    def __init__(self,
                 data: dict,
                 transform: tr.Compose,
                 client_name: str):
        super().__init__()
        self.samples = [(image, label) for image, label in zip(data['img'], data['class'])]
        self.transform = transform
        self.client_name = client_name


    def __getitem__(self, index: int):
        sample = self.samples[index]
        image = Image.fromarray(np.uint8(sample[0].reshape(28,28)*255))
        label = sample[1]


        if self.transform is not None:

            image = self.transform(image)

        
        return image, label

    def __len__(self) -> int:
        return len(self.samples)

In [13]:
## test

obj = Femnist(batch, convert_tensor, None)
obj.__getitem__(1)[0].shape

torch.Size([1, 28, 28])

In [14]:
batch.loc[1,'img'].shape

(784,)

In [15]:
Image.fromarray(np.uint8(batch.loc[1,'img'].reshape(28,28)*255))

In [36]:
class CNN_1(nn.Module): 
    def __init__(self, num_classes): 
        super(CNN_1, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=32, 
                               kernel_size=5, 
                               stride=1
                               ) 

        self.conv2 = nn.Conv2d(in_channels=32, 
                               out_channels=64, 
                               kernel_size=5, 
                               stride=1
                               ) 
        self.fc1 = nn.Linear(64 * 5 * 5, 2048) 
        self.fc2 = nn.Linear(2048, num_classes) 
    
    def forward(self, x):
        x =  F.relu(
                    F.max_pool2d(
                      self.conv1(x),2,2
                      )
                    )
        x =  F.relu(
                   F.max_pool2d(
                      self.conv2(x),2,2
                      )
                   ) 
        # x =  x.view(-1,x.size(0)) 
        x =  F.relu(self.fc1(x))
        x =  self.fc2(x) 
        return x


In [37]:
class CNN_2(nn.Module):
    def __init__(self, num_classes):
        super(CNN_2, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 2048)
        self.fc2 = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 7 * 7 * 64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [40]:
X_train, X_val, y_train, y_val = train_test_split(batch['img'], batch['class'], test_size=0.2, random_state=42)

torch_train = Femnist(pd.DataFrame(
    {'img': X_train.tolist(),
     'class': y_train.tolist()}), convert_tensor, None
)


torch_test = Femnist(pd.DataFrame(
    {'img': X_val.tolist(),
     'class': y_val.tolist()}), convert_tensor, None
)
                        


In [41]:
X_train

607753    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
396266    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
367753    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
620298    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
662182    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
                                ...                        
259178    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
365838    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
131932    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
671155    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
121958    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
Name: img, Length: 654280, dtype: object

In [20]:
print("length of the dataset is:", len(torch_train))

length of the dataset is: 654280


In [46]:
n_classes = batch['class'].unique().shape[0]

train_loader = DataLoader(torch_train, batch_size=64, shuffle=True) 
val_loader = DataLoader(torch_test, batch_size=64, shuffle=False) # set up model and optimizer 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
CNNmodel = CNN_2(n_classes).to(device)
optimizer = torch.optim.SGD(CNNmodel.parameters(), lr=0.001) # define loss function criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = CNNmodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 3.745
[1,  4000] loss: 3.676


In [ ]:
PATH = './central_net_5epochs.pth'
torch.save(CNNmodel.state_dict(), PATH)

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = CNNmodel(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = CNNmodel(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [31]:
#CNNmodel.state_dict()